In [30]:
import pandas as pd 
import numpy as np 
import joblib
from datetime import datetime
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn import metrics
collection_of_model = [LogisticRegression(),RandomForestClassifier(),
                       AdaBoostClassifier(),
                       XGBClassifier(),KNeighborsClassifier(),LGBMClassifier()]

TARGET = 'SeriousDlqin2yrs'
TRAINING_PATH = "../data/interim/interim_training_forth_exp_tuned.csv"

In [31]:
#read data 
training_data = pd.read_csv(TRAINING_PATH)
#split data into X and Y
X = training_data.drop(TARGET,axis=1)
Y = training_data[TARGET]

In [32]:
#define cross validation 

fold = StratifiedKFold(5)
def begin_cross_validation(X,Y,model,fold=fold) : 
    scoring_dict = {'auc':[]}
    for train_idx,test_idx in fold.split(X,Y) : 
        train_x,val_x = X.iloc[train_idx],X.iloc[test_idx]
        train_y,val_y = Y.iloc[train_idx],Y.iloc[test_idx]
        #train model 

        model.fit(train_x,train_y)
        y_pred = model.predict_proba(val_x)
        #calculate metrics 
        scoring_dict["auc"].append(metrics.roc_auc_score(val_y, y_pred[:, 1]))


    #print the scoring 
    print(f'''Scoring: AUC :{np.mean(scoring_dict['auc'])}
     ''')
    return np.mean(scoring_dict['auc'])

In [6]:
begin_cross_validation

In [7]:

for model in collection_of_model : 
    model_name = type(model).__name__
    print(model_name)
    print('========')
    start_time = datetime.now()
    container_dict[model_name] = {}
    auc_cv = begin_cross_validation(X,Y,model=model)
    end_time = datetime.now()
    fit_time = end_time-start_time
    container_dict[model_name]['time']=fit_time
    container_dict[model_name]['auc']=auc_cv

LogisticRegression


/home/fakhri/pacmann_project/project_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/fakhri/pacmann_project/project_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/mod

Scoring: AUC :0.7404727184910505
     
RandomForestClassifier
Scoring: AUC :0.8400384485751395
     
AdaBoostClassifier
Scoring: AUC :0.8589782322542809
     
XGBClassifier
Scoring: AUC :0.8580848211772993
     
KNeighborsClassifier
Scoring: AUC :0.690178270390196
     
LGBMClassifier
Scoring: AUC :0.8645873614338461
     


In [8]:
joblib.dump(container_dict,"model_comparison.joblib")

['model_comparison.joblib']

In [9]:
training_records = joblib.load('model_comparison.joblib')

In [19]:
recap = pd.DataFrame(training_records).T.reset_index()

In [20]:
recap = recap.rename(columns={'index':'model'})

In [21]:
recap.style.highlight_max(color = 'green',subset='auc')

,model,time,auc
0,LogisticRegression,0 days 00:00:02.613927,0.740473
1,RandomForestClassifier,0 days 00:01:40.210478,0.840038
2,AdaBoostClassifier,0 days 00:00:19.730037,0.858978
3,XGBClassifier,0 days 00:00:20.310368,0.858085
4,KNeighborsClassifier,0 days 00:00:22.466657,0.690178
5,LGBMClassifier,0 days 00:00:01.906390,0.864587


In [22]:
recap.style.highlight_min(color = 'green',subset='time')

,model,time,auc
0,LogisticRegression,0 days 00:00:02.613927,0.740473
1,RandomForestClassifier,0 days 00:01:40.210478,0.840038
2,AdaBoostClassifier,0 days 00:00:19.730037,0.858978
3,XGBClassifier,0 days 00:00:20.310368,0.858085
4,KNeighborsClassifier,0 days 00:00:22.466657,0.690178
5,LGBMClassifier,0 days 00:00:01.906390,0.864587


In [23]:
filter_ = (recap['index']=="LogisticRegression") 
np.mean(recap.loc[filter_,'auc']['auc'])

KeyError: 'auc'

In [25]:
params = {  "lambda_l1": 2.8611070127189538e05 ,
  "lambda_l2": 0.6807987664418726 ,
  "num_leaves": 9 ,
  "max_depth": 19 ,
  "feature_fraction": 0.5332191589953184 ,
  "bagging_fraction": 0.9238892410770332 ,
  "bagging_freq": 3 ,
  "min_child_samples": 73,
         }



clf_model = LGBMClassifier(**params).fit(X,Y)

[LightGBM] [Warning] lambda_l1 is set=286110.7012718954, reg_alpha=0.0 will be ignored. Current value: lambda_l1=286110.7012718954
[LightGBM] [Warning] feature_fraction is set=0.5332191589953184, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5332191589953184
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l2 is set=0.6807987664418726, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.6807987664418726
[LightGBM] [Warning] bagging_fraction is set=0.9238892410770332, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9238892410770332
